# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from bokeh.models import HoverTool

# Import API key
from api_keys import geoapify_key

In [25]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,299.12,77,40,6.17,US,1686772923
1,1,grytviken,-54.2811,-36.5092,270.20,82,45,2.98,GS,1686772758
2,2,vila velha,-20.3297,-40.2925,299.12,81,20,5.66,BR,1686772730
3,3,zwedru,6.0667,-8.1281,296.09,92,97,1.61,LR,1686772924
4,4,bilibino,68.0546,166.4372,275.59,58,93,1.28,RU,1686772925


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    scale = 0.5,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
#       my ideal weather condition is: cloudiness < 40%, humidity < 50%
# Drop any rows with null values

ideal_city_data_df = city_data_df[(city_data_df['Cloudiness'] < 40) & (city_data_df['Humidity'] < 50)]

# Display sample data
ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,palmas,-10.2128,-48.3603,306.08,43,0,3.60,BR,1686772726
21,21,al jahra',29.3375,47.6581,308.09,19,0,5.22,KW,1686772933
22,22,karratha,-20.7377,116.8463,292.86,38,13,4.42,AU,1686772933
39,39,al burayqah,30.4062,19.5739,302.99,38,0,7.72,LY,1686772943
74,74,bloemfontein,-29.1211,26.2140,284.86,37,0,3.60,ZA,1686772829


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
12,palmas,BR,-10.2128,-48.3603,43,
21,al jahra',KW,29.3375,47.6581,19,
22,karratha,AU,-20.7377,116.8463,38,
39,al burayqah,LY,30.4062,19.5739,38,
74,bloemfontein,ZA,-29.1211,26.2140,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
limit = 20


params = {
    "categories": "accommodation.hotel",
    "limit": limit,
    "apiKey": geoapify_key
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
palmas - nearest hotel: Hotel Triângulo Mineiro
al jahra' - nearest hotel: Al Jahra Copthorne Hotel & Resort
karratha - nearest hotel: Karratha International Hotel
al burayqah - nearest hotel: فندق ماس ليبيا
bloemfontein - nearest hotel: Stadium Inn
mount isa - nearest hotel: Ibis Styles
at taj - nearest hotel: No hotel found
kuraymah - nearest hotel: Hotel al Nasser
beroroha - nearest hotel: No hotel found
torata - nearest hotel: No hotel found
kabompo - nearest hotel: No hotel found
tokar - nearest hotel: No hotel found
safidon - nearest hotel: No hotel found
ar rutbah - nearest hotel: No hotel found
ha'il - nearest hotel: فندق كلاسيك
khanty-mansiysk - nearest hotel: Гамма
kill devil hills - nearest hotel: Mariner Days Inn & Suites
zouerate - nearest hotel: فندق تيرس زمور
atar - nearest hotel: فندق سكليل
sarandi del yi - nearest hotel: No hotel found
lycksele - nearest hotel: City Hotellet


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(x='Lng', y='Lat', geo = True,
    tiles = "OSM", hover_cols=['City', 'Hotel Name'], size=10, color='red')

# Customize the hover tooltip
hover_tooltips = [
    ('City', '@City'),
    ('Hotel Name', '@{Hotel Name}')
]
hover = HoverTool(tooltips=hover_tooltips)
map_plot = map_plot.opts(tools=[hover])

# Display the map
map_plot